In [18]:
# Cargar modelo
# ==============================================================================
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import pickle
import h2o
from h2o.automl import H2OAutoML
# inicializar H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,44 mins 30 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_juano_qk5ryf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.119 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [19]:
# Cargamos los df para el modelo
# ===============================================================================

df = pd.read_csv('../data/merged_df_2022.csv',index_col= 0)
df_train=df.iloc[1:].to_csv('../data/df_train.csv')
df_test= (df.iloc[:1]).drop('revenue_resold', axis=1).to_csv('../data/df_test.csv')

In [21]:
# Cargar los datos en un H2OFrame
# ===============================================================================

data = h2o.import_file('../data/df_train.csv')

# separar el conjunto de datos en entrenamiento y prueba
train, test = data.split_frame(ratios=[0.8])

# definir la variable objetivo y las variables predictoras
y = 'revenue_resold'
x = data.col_names
x.remove(y)

# ejecutar H2O AutoML
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# obtener los mejores modelos
lb = aml.leaderboard
print(lb)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |█
20:16:30.155: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
model_id                                                    rmse      mse       mae      rmsle    mean_residual_deviance
DeepLearning_grid_2_AutoML_2_20230421_201630_model_1     48.7322  2374.83  12.7769   nan                         2374.83
StackedEnsemble_AllModels_1_AutoML_2_20230421_201630     49.2501  2425.57  10.6475   nan                         2425.57
StackedEnsemble_BestOfFamily_1_AutoML_2_20230421_201630  51.596   2662.14  10.8638   nan                         2662.14
GBM_grid_1_AutoML_2_20230421_201630_model_2              60.2721  3632.72   6.95351  nan                         3632.72
DeepLearning_grid_3_AutoML_2_20230421_201630_model_1     63.4097  4020.79  17.7381     1.95121                   4020.79
GBM_5_AutoML_2_202304

In [22]:
# Obtener el mejor modelo del AutoML y guardarlo
# ==============================================================================

best_model = aml.leader
model_path = h2o.save_model(model=best_model, path="../data/best_model_3", force=True)

In [39]:
# Cargar el mejor modelo en memoria
# ==============================================================================

loaded_model = h2o.load_model(model_path)

In [25]:
# Predecimos 
# ==============================================================================

# Cargar los nuevos datos de entrada como un objeto H2OFrame
new_data = h2o.import_file(path = "../data/df_test.csv")

# Hacer predicciones en los nuevos datos utilizando el modelo cargado
predictions = loaded_model.predict(new_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [26]:
# Convertir las predicciones a un objeto pandas DataFrame y guardarlo como un archivo CSV
# ==============================================================================

predictions_df = predictions.as_data_frame()

In [37]:
# Printeamos el resultado de la predicción
# ==============================================================================

print(predictions_df['predict'][0].round(2))

16727.99


In [31]:
# Printeamos la info del mejor modelo
# ==============================================================================

print(aml.leader)

Model Details
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_2_20230421_201630_model_1


Status of Neuron Layers: predicting revenue_resold, regression, gaussian distribution, Quadratic loss, 19.501 weights/biases, 239,2 KB, 14.303.520 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate               rate_rms                momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ----------------------  ----------------------  ----------  --------------------  -------------------  ---------------------  -----------------------
    1        92       Input             15.0
    2        100      RectifierDropout  10.0       0.0   0.0   0.189901159426677       0.31386053562164307     0.0         0.043827192973419936  1.1007370948791504   -0.005354712631654974  1.9661340713500977
    3        100      RectifierD